# WildTrack AI - Data PreProcessing

This notebook is used to preprocess images per specifications for various models that were experimented with for Species Classification and Individual Identification tasks

## 1. Initialization

In [0]:
#Set up TF 2.0
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [6]:
#Import Libraries

#General
import csv
import os
import numpy as np
from collections import defaultdict
import random
import pickle

# Image processing libraries
from keras.preprocessing import image as KImage
import cv2
from google.colab.patches import cv2_imshow


# Preprocessors from Pretrained Models

from tensorflow.keras.applications.vgg16 import preprocess_input as VGG16Pre
from tensorflow.keras.applications.vgg19 import preprocess_input as VGG19Pre
from tensorflow.keras.applications.inception_v3 import preprocess_input as InceptionPre
from tensorflow.keras.applications.xception import preprocess_input as XceptionPre
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as MNPre
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# Plotting 
%matplotlib inline
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [3]:
#Mount Google Drive - Note this mounts your personal GDrive to the directory stated
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 2. Image Load and Pre-Process

Images are loaded, pre-processed and saved back to csv files (one line per image) for retrieval later for Species Classification

In [7]:
#Set up path for training/ test data. Need to map U C Berkeley - Darragh/ shared to your personal GDrive for this to work. 
path="/content/drive/My Drive/U C Berkeley - Darragh/Training Data"
test_path="/content/drive/My Drive/U C Berkeley - Darragh/Test Data"
csvpath="/content/drive/My Drive/U C Berkeley - Darragh/csv"
os.listdir(path)

['Amur Tiger',
 'Bengal Tiger',
 'Cheetah',
 'Leopard',
 'Lowland Tapir',
 'Puma',
 'White Rhino',
 'Black Rhino',
 'African lion',
 'African elephant',
 'Bongo']

In [0]:
# Function to load and individual image to a specified size.
def temp_load_image(image,preprocessor,size=(299,299)):
  try:
    print(image)
    image = KImage.load_img(image,target_size=size,interpolation="nearest")

  except:
    return np.zeros(0),np.zeros(0)
  else:
    raw= KImage.img_to_array(image)
    x = np.expand_dims(raw, axis=0)
    x = preprocessor(x)
    x=np.squeeze(x)
    #x=x.reshape(-1)
    return image,x

In [9]:
#Test One Image
raw,x=temp_load_image(os.path.join(test_path,'Cheetah','Aiko','Aiko.JPG'),VGG16Pre,(224,224))
opencvImage = cv2.cvtColor(np.array(raw), cv2.COLOR_RGB2BGR)
plt.imshow(raw)

/content/drive/My Drive/U C Berkeley - Darragh/Test Data/Cheetah/Aiko/Aiko.JPG


In [0]:
# Function to load and individual image to a specified size.
def load_image(image,preprocessor,size=(299,299)):
  try:
    image = KImage.load_img(image,target_size=size,interpolation="nearest")
  except:
    return np.zeros(0)
  else:
    x= KImage.img_to_array(image)
    x = np.expand_dims(x, axis=0)
    x = preprocessor(x)
    x=np.squeeze(x)
    x=x.reshape(-1)
    return x

In [0]:
# Function to load image data set from specified folder. Will be reshaped to size specified in size parameter.
# Once images are processed, writes back out to a csv file

def LoadDataSet(folder,img_output,label_output,preprocessor,size=(224,224)):
  Keys=[]
  with open(img_output, 'w') as f:
    writer = csv.writer(f)
    for species in os.listdir(folder):
      ind_path=folder+'/'+species
      print(species)

      for individual in os.listdir(ind_path):
        print_path=ind_path+'/'+individual

        for footprint in os.listdir(print_path):
          if footprint.find(" RH ")>0:
            continue
          else:
            x=load_image(os.path.join(print_path,footprint),preprocessor,(224,224))
            if x.shape[0]==0:
              continue
            else:
              key=species+"-"+individual
              Keys.append(key)
              writer.writerow(x)

  with open(label_output, 'w') as filehandle:
    filehandle.writelines("%s\n" % key for key in Keys)

In [15]:
#Point to path to write csv images and create pre-processed training and test input for VGG16 model (224x224 image size) 

LoadDataSet(path,os.path.join(csvpath,"Training-Images-224.csv"),os.path.join(csvpath,"Training-Labels-224.txt"),VGG16Pre,(224,224))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-224.csv"),os.path.join(csvpath,"Test-Labels-224.txt"),VGG16Pre,(224,224))

Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo


In [0]:
#Create similar csv files for Mobilenet (224x224) and Inception net (299x299)

LoadDataSet(path,os.path.join(csvpath,"Training-Images-299.csv"),os.path.join(csvpath,"Training-Labels-299.txt"),preprocess_input,(299,299))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-299.csv"),os.path.join(csvpath,"Test-Labels-299.txt"),preprocess_input,(299,299))
LoadDataSet(path,os.path.join(csvpath,"Train-Images-Mobile-224.csv"),os.path.join(csvpath,"Train-Labels-Mobile-224.txt"),MNPre,(224,224))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-Mobile-224.csv"),os.path.join(csvpath,"Test-Labels-Mobile-224.txt"),MNPre,(224,224))

Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo


In [0]:
#csv files for Xception, VGG19 and Inception

LoadDataSet(path,os.path.join(csvpath,"Training-Images-Xception-224.csv"),os.path.join(csvpath,"Training-Labels-Xception-224.txt"),XceptionPre,(299,299))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-Xception-224.csv"),os.path.join(csvpath,"Test-Labels-Xception-224.txt"),XceptionPre,(299,299))
LoadDataSet(path,os.path.join(csvpath,"Training-Images-VGG19-224.csv"),os.path.join(csvpath,"Training-Labels-VGG19-224.txt"),VGG19Pre,(224,224))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-VGG19-224.csv"),os.path.join(csvpath,"Test-Labels-VGG19-224.txt"),VGG19Pre,(224,224))
LoadDataSet(path,os.path.join(csvpath,"Training-Images-Inception-299.csv"),os.path.join(csvpath,"Training-Labels-Inception-299.txt"),InceptionPre,(299,299))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-Inception-299.csv"),os.path.join(csvpath,"Test-Labels-Inception-299.txt"),InceptionPre,(299,299))

Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo


In [0]:
#csv files for EfficientNET
LoadDataSet(path,os.path.join(csvpath,"Training-Images-EfficientB4-224.csv"),os.path.join(csvpath,"Training-Labels-EfficientB4-224.txt"),XceptionPre,(299,299))
LoadDataSet(test_path,os.path.join(csvpath,"Test-Images-EfficientB4-224.csv"),os.path.join(csvpath,"Test-Labels-EfficientB4-224.txt"),XceptionPre,(299,299))

Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
Amur Tiger
Bengal Tiger
Cheetah
Leopard
Lowland Tapir
Puma
White Rhino
Black Rhino
African lion
African elephant
Bongo
